In [6]:
from functools import partial
import itertools
import json
import re

import numpy as np
import scipy as sp
import sklearn
import sklearn.metrics
from sklearn.utils import check_random_state

def __data_labels_distances(
                                indexed_string,
                                classifier_fn,
                                num_samples,
                                distance_metric='cosine'):

    def distance_fn(x):
        return sklearn.metrics.pairwise.pairwise_distances(
            x, x[0], metric=distance_metric).ravel() * 100

    doc_size = indexed_string.num_words()
    sample = random_state.randint(1, doc_size + 1, num_samples - 1)
    data = np.ones((num_samples, doc_size))
    data[0] = np.ones(doc_size)
    features_range = range(doc_size)
    inverse_data = [indexed_string.raw_string()]
    for i, size in enumerate(sample, start=1):
        inactive = random_state.choice(features_range, size,
                                            replace=False)
        data[i, inactive] = 0
        inverse_data.append(indexed_string.inverse_removing(inactive))
    #labels = classifier_fn(inverse_data)
    distances = distance_fn(sp.sparse.csr_matrix(data))
    #return data, labels, distances
    return data, distances

In [8]:
distances = __data_labels_distances(indexed_string=indexed_string, classifier_fn=None, num_samples=3)

In [9]:
d1 = distances[0]
d2 = distances[1]

In [49]:
d1[0]

array([1., 1., 1., ..., 1., 1., 1.])

In [4]:
with open("textcomb4.txt", 'r') as text:
    text_instance = text.read()

In [7]:
kernel_width=25,
kernel=None,
verbose=False,
class_names=None,
feature_selection='auto',
split_expression=r'\W+',
bow=True,
mask_string=None,
char_level=False
random_state = check_random_state(None)

In [13]:
indexed_string = (MyIndexedString(text_instance, bow=bow,
                                        split_expression=split_expression,
                                        mask_string=mask_string))

1 0
1 1
1 1
1 2
1 3
1 4
1 5
1 6
1 7
1 0
1 8
2 0
2 1
2 2
2 3
2 4
2 5
2 6
2 7
2 3
2 8
2 9


In [1]:
text_instance = 'Rinoa let let out a soft giggle. "Okay Uncle Rinoa Laguna." $&*&*&$"As always, make a giggle Rinoa yourselves at a good home!" '

In [9]:
print(vocab, vocab_right)

{'Rinoa': 0, 'let': 1, 'out': 2, 'a': 3, 'soft': 4, 'giggle': 5, 'Okay': 6, 'Uncle': 7, 'Laguna': 8} {'As': 0, 'always': 1, 'make': 2, 'a': 3, 'giggle': 4, 'Rinoa': 5, 'yourselves': 6, 'at': 7, 'good': 8, 'home': 9}


In [60]:
l = indexed_string
print(l.positions, l.sep, l.positions_right, l.num_words())
print(l.inverse_vocab, l.inverse_vocab_right)

[[0, 18], [2, 4], [6], [8], [10], [12], [14], [16], [20]] 9 [[22], [24], [26], [28, 38], [30], [32], [34], [36], [40], [42]] 19
['Rinoa', 'let', 'out', 'a', 'soft', 'giggle', 'Okay', 'Uncle', 'Laguna'] ['As', 'always', 'make', 'a', 'giggle', 'Rinoa', 'yourselves', 'at', 'good', 'home']


In [15]:
distances = __data_labels_distances(indexed_string=indexed_string, classifier_fn=None, num_samples=3, mode='left')

sep:  9 docsize:  19
left bow
sample:  [8 5]
1 9 8
inactive [4 2 5 1 6 7 8 3]
2 9 5
inactive [3 5 0 1 4]
['Rinoa let let out a soft giggle. "Okay Uncle Rinoa Laguna." $&*&*&$"As always, make a giggle Rinoa yourselves at a good home!" ', 'Rinoa      . "  Rinoa ." $&*&*&$"As always, make a giggle Rinoa yourselves at a good home!" ', '   out   . "Okay Uncle  Laguna." $&*&*&$"As always, make a giggle Rinoa yourselves at a good home!" ']


In [16]:
print(distances[0])

[[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [0. 0. 1. 0. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]]


In [12]:
def __data_labels_distances(
                                indexed_string,
                                classifier_fn,
                                num_samples,
                                distance_metric='cosine',
                                mode='rand'):
        """Generates a neighborhood around a prediction.
        Generates neighborhood data by randomly removing words from
        the instance, and predicting with the classifier. Uses cosine distance
        to compute distances between original and perturbed instances.
        Args:
            indexed_string: document (IndexedString) to be explained,
            classifier_fn: classifier prediction probability function, which
                takes a string and outputs prediction probabilities. For
                ScikitClassifier, this is classifier.predict_proba.
            num_samples: size of the neighborhood to learn the linear model
            distance_metric: the distance metric to use for sample weighting,
                defaults to cosine similarity.
        Returns:
            A tuple (data, labels, distances), where:
                data: dense num_samples * K binary matrix, where K is the
                    number of tokens in indexed_string. The first row is the
                    original instance, and thus a row of ones.
                labels: num_samples * L matrix, where L is the number of target
                    labels
                distances: cosine distance between the original instance and
                    each perturbed instance (computed in the binary 'data'
                    matrix), times 100.
        """
        def distance_fn(x):
            return sklearn.metrics.pairwise.pairwise_distances(
                    x, x[0], metric=distance_metric).ravel() * 100


        doc_size = indexed_string.num_words()

        sep = indexed_string.return_sep()
        print("sep: ", sep, "docsize: ", doc_size)

        global sample
        #
        sample = random_state.randint(1, doc_size + 1, num_samples - 1)

        data = np.ones((num_samples, doc_size))
        data[0] = np.ones(doc_size)
        inverse_data = [indexed_string.raw_string()]

        if not indexed_string.bow: 
            
            if sep < doc_size:
                print("separation on")
                #
                #
                sample_left = random_state.randint(1, sep + 1, num_samples - 1)
                sample_right = random_state.randint(1, doc_size - sep, num_samples - 1)

                features_range_left = range(0, sep)
                features_range_right = range(sep, doc_size)

                if mode == 'left':
                    print("left")
                    sample = sample_left
                    features_range = features_range_left

                    #global save_sample
                    #save_sample = sample

                    for i, size in enumerate(sample, start=1):
                        #
                        inactive = random_state.choice(features_range, size,
                                                        replace=False)
                        data[i, inactive] = 0
                        inverse_data.append(indexed_string.inverse_removing(inactive))

                elif mode == 'right':
                    print("right")
                    sample = sample_right
                    features_range = features_range_right

                    #global save_sample
                    #save_sample = sample

                    for i, size in enumerate(sample, start=1):
                        print(i, len(features_range), size)
                        #
                        inactive = random_state.choice(features_range, size,
                                                        replace=False)
                        data[i, inactive] = 0
                        inverse_data.append(indexed_string.inverse_removing(inactive))

                else:
                    print('rand')
                    dir = self.random_state.randint(2, size = num_samples - 1)

                    for i, size in enumerate(sample, start=1):
                        #print(dir[i-1])
                        if dir[i-1]:
                            #
                            inactive = random_state.choice(features_range_right, sample_right[i-1],
                                                        replace=False)
                        else:
                            #
                            inactive = random_state.choice(features_range_left, sample_left[i-1],
                                                        replace=False)
                        data[i, inactive] = 0
                        inverse_data.append(indexed_string.inverse_removing(inactive))

            else:
                features_range = range(doc_size)
                for i, size in enumerate(sample, start=1):
                    #
                    inactive = random_state.choice(features_range, size,
                                                        replace=False)
                    data[i, inactive] = 0
                    inverse_data.append(indexed_string.inverse_removing(inactive))            

        else:

            if indexed_string.positions_right:
                #
                #
                sample_left = random_state.randint(1, sep + 1, num_samples - 1)
                sample_right = random_state.randint(1, doc_size - sep, num_samples - 1)

                features_range_left = range(0, sep)
                features_range_right = range(0, doc_size - sep)

                global save_sample
                save_sample = sample

                if mode == 'left':
                    print("left bow")
                    sample = sample_left
                    print("sample: ", sample)
                    features_range = features_range_left

                elif mode == 'right':
                    print("right bow")
                    sample = sample_right
                    print("sample: ", sample)
                    features_range = features_range_right

                for i, size in enumerate(sample, start=1):
                    print(i, len(features_range), size)
                    #
                    inactive = random_state.choice(features_range, size,
                                                    replace=False)
                    data[i, inactive] = 0
                    print('inactive', inactive)
                    inverse_data.append(indexed_string.inverse_removing(inactive, mode=mode))

            else:
                features_range = range(doc_size)
                for i, size in enumerate(sample, start=1):
                    #
                    inactive = random_state.choice(features_range, size,
                                                        replace=False)
                    data[i, inactive] = 0
                    inverse_data.append(indexed_string.inverse_removing(inactive))

        #labels = classifier_fn(inverse_data)
        print(inverse_data)
        distances = distance_fn(sp.sparse.csr_matrix(data))
        #return data, labels, distances
        return data, distances

In [2]:
class MyIndexedString(object):
    """String with various indexes."""

    def __init__(self, raw_string, split_expression=r'\W+', bow=True,
                 mask_string=None):
        """Initializer.
        Args:
            raw_string: string with raw text in it
            split_expression: Regex string or callable. If regex string, will be used with re.split.
                If callable, the function should return a list of tokens.
            bow: if True, a word is the same everywhere in the text - i.e. we
                 will index multiple occurrences of the same word. If False,
                 order matters, so that the same word will have different ids
                 according to position.
            mask_string: If not None, replace words with this if bow=False
                if None, default value is UNKWORDZ
        """
        self.raw = raw_string
        self.mask_string = 'UNKWORDZ' if mask_string is None else mask_string

        if callable(split_expression):
            tokens = split_expression(self.raw)
            self.as_list = self._segment_with_tokens(self.raw, tokens)
            tokens = set(tokens)

            def non_word(string):
                return string not in tokens

        else:
            # with the split_expression as a non-capturing group (?:), we don't need to filter out
            # the separator character from the split results.
            splitter = re.compile(r'(%s)|$' % split_expression)
            self.as_list = [s for s in splitter.split(self.raw) if s]
            non_word = splitter.match

        self.as_np = np.array(self.as_list)
        self.string_start = np.hstack(
            ([0], np.cumsum([len(x) for x in self.as_np[:-1]])))

        global vocab

        vocab = {}
        self.inverse_vocab = []
        self.positions = []
        self.bow = bow

        self.sep = None


        #added in lime22
        self.positions_right = []
        self.inverse_vocab_right = []
        global vocab_right
        vocab_right = {}

        non_vocab = set()
        for i, word in enumerate(self.as_np):
            if "$&*&*&$" in word:
                self.sep = len(self.inverse_vocab)
            if word in non_vocab:
                continue
            if non_word(word):
                non_vocab.add(word)
                continue
            if bow:

                """if word not in vocab:
                    vocab[word] = len(vocab)
                    self.inverse_vocab.append(word)
                    self.positions.append([])
                idx_word = vocab[word]
                self.positions[idx_word].append(i)"""

                if not self.sep:
                    if word not in vocab:
                        vocab[word] = len(vocab)
                        self.inverse_vocab.append(word)
                        self.positions.append([])
                    idx_word = vocab[word]
                    print(1, idx_word)
                    self.positions[idx_word].append(i)
                else:
                    if word not in vocab_right:
                        vocab_right[word] = len(vocab_right)
                        self.inverse_vocab_right.append(word)
                        self.positions_right.append([])
                    idx_word = vocab_right[word]
                    print(2, idx_word)
                    self.positions_right[idx_word].append(i)

            else:
                self.inverse_vocab.append(word)
                self.positions.append(i)

        if not bow:
            self.positions = np.array(self.positions)

        if not self.sep:
            self.sep = len(self.inverse_vocab)

    def return_sep(self):
        """Return the index of the separator sequence"""
        return self.sep

    def raw_string(self):
        """Returns the original raw string"""
        return self.raw

    def num_words(self):
        """Returns the number of tokens in the vocabulary for this document."""
        return len(self.inverse_vocab) if not self.inverse_vocab_right else len(self.inverse_vocab) + len(self.inverse_vocab_right)

    def word(self, id_):
        """Returns the word that corresponds to id_ (int)"""
        return self.inverse_vocab[id_]

    def string_position(self, id_):
        """Returns a np array with indices to id_ (int) occurrences"""
        if self.bow:
            return self.string_start[self.positions[id_]]
        else:
            return self.string_start[[self.positions[id_]]]

    def inverse_removing(self, words_to_remove, mode='left'):
        """Returns a string after removing the appropriate words.
        If self.bow is false, replaces word with UNKWORDZ instead of removing
        it.
        Args:
            words_to_remove: list of ids (ints) to remove
        Returns:
            original raw string with appropriate words removed.
        """
        mask = np.ones(self.as_np.shape[0], dtype='bool')
        if mode == "left":
            mask[self.__get_idxs(words_to_remove)] = False
        elif mode == "right":
            mask[self.__get_idxs_right(words_to_remove)] = False
        if not self.bow:
            return ''.join(
                [self.as_list[i] if mask[i] else self.mask_string
                 for i in range(mask.shape[0])])
        return ''.join([self.as_list[v] for v in mask.nonzero()[0]])

    @staticmethod
    def _segment_with_tokens(text, tokens):
        """Segment a string around the tokens created by a passed-in tokenizer"""
        list_form = []
        text_ptr = 0
        for token in tokens:
            inter_token_string = []
            while not text[text_ptr:].startswith(token):
                inter_token_string.append(text[text_ptr])
                text_ptr += 1
                if text_ptr >= len(text):
                    raise ValueError("Tokenization produced tokens that do not belong in string!")
            text_ptr += len(token)
            if inter_token_string:
                list_form.append(''.join(inter_token_string))
            list_form.append(token)
        if text_ptr < len(text):
            list_form.append(text[text_ptr:])
        return list_form

    def __get_idxs(self, words):
        """Returns indexes to appropriate words."""
        if self.bow:
            return list(itertools.chain.from_iterable(
                [self.positions[z] for z in words]))
        else:
            return self.positions[words]

    def __get_idxs_right(self, words):
        """Returns indexes to appropriate words."""
        if self.bow:
            return list(itertools.chain.from_iterable(
                [self.positions_right[z] for z in words]))
        else:
            return self.positions_right[words]

# Old code to compare!

In [ ]:
class OldMyIndexedString(object):
    """String with various indexes."""

    def __init__(self, raw_string, split_expression=r'\W+', bow=True,
                 mask_string=None):
        """Initializer.
        Args:
            raw_string: string with raw text in it
            split_expression: Regex string or callable. If regex string, will be used with re.split.
                If callable, the function should return a list of tokens.
            bow: if True, a word is the same everywhere in the text - i.e. we
                 will index multiple occurrences of the same word. If False,
                 order matters, so that the same word will have different ids
                 according to position.
            mask_string: If not None, replace words with this if bow=False
                if None, default value is UNKWORDZ
        """
        self.raw = raw_string
        self.mask_string = 'UNKWORDZ' if mask_string is None else mask_string

        if callable(split_expression):
            tokens = split_expression(self.raw)
            self.as_list = self._segment_with_tokens(self.raw, tokens)
            tokens = set(tokens)

            def non_word(string):
                return string not in tokens

        else:
            # with the split_expression as a non-capturing group (?:), we don't need to filter out
            # the separator character from the split results.
            splitter = re.compile(r'(%s)|$' % split_expression)
            self.as_list = [s for s in splitter.split(self.raw) if s]
            non_word = splitter.match

        self.as_np = np.array(self.as_list)
        self.string_start = np.hstack(
            ([0], np.cumsum([len(x) for x in self.as_np[:-1]])))
        vocab = {}
        self.inverse_vocab = []
        self.positions = []
        self.bow = bow

        self.sep = None

        non_vocab = set()
        for i, word in enumerate(self.as_np):
            if "$&*&*&$" in word:
                self.sep = len(self.inverse_vocab)
            if word in non_vocab:
                continue
            if non_word(word):
                non_vocab.add(word)
                continue
            if bow:
                if word not in vocab:
                    vocab[word] = len(vocab)
                    self.inverse_vocab.append(word)
                    self.positions.append([])
                idx_word = vocab[word]
                self.positions[idx_word].append(i)
            else:
                self.inverse_vocab.append(word)
                self.positions.append(i)
        if not bow:
            self.positions = np.array(self.positions)

        if not self.sep:
            self.sep = len(self.inverse_vocab)

    def return_sep(self):
        """Return the index of the separator sequence"""
        return self.sep

    def raw_string(self):
        """Returns the original raw string"""
        return self.raw

    def num_words(self):
        """Returns the number of tokens in the vocabulary for this document."""
        return len(self.inverse_vocab)

    def word(self, id_):
        """Returns the word that corresponds to id_ (int)"""
        return self.inverse_vocab[id_]

    def string_position(self, id_):
        """Returns a np array with indices to id_ (int) occurrences"""
        if self.bow:
            return self.string_start[self.positions[id_]]
        else:
            return self.string_start[[self.positions[id_]]]

    def inverse_removing(self, words_to_remove):
        """Returns a string after removing the appropriate words.
        If self.bow is false, replaces word with UNKWORDZ instead of removing
        it.
        Args:
            words_to_remove: list of ids (ints) to remove
        Returns:
            original raw string with appropriate words removed.
        """
        mask = np.ones(self.as_np.shape[0], dtype='bool')
        mask[self.__get_idxs(words_to_remove)] = False
        if not self.bow:
            return ''.join(
                [self.as_list[i] if mask[i] else self.mask_string
                 for i in range(mask.shape[0])])
        return ''.join([self.as_list[v] for v in mask.nonzero()[0]])

    @staticmethod
    def _segment_with_tokens(text, tokens):
        """Segment a string around the tokens created by a passed-in tokenizer"""
        list_form = []
        text_ptr = 0
        for token in tokens:
            inter_token_string = []
            while not text[text_ptr:].startswith(token):
                inter_token_string.append(text[text_ptr])
                text_ptr += 1
                if text_ptr >= len(text):
                    raise ValueError("Tokenization produced tokens that do not belong in string!")
            text_ptr += len(token)
            if inter_token_string:
                list_form.append(''.join(inter_token_string))
            list_form.append(token)
        if text_ptr < len(text):
            list_form.append(text[text_ptr:])
        return list_form

    def __get_idxs(self, words):
        """Returns indexes to appropriate words."""
        if self.bow:
            return list(itertools.chain.from_iterable(
                [self.positions[z] for z in words]))
        else:
            return self.positions[words]

In [ ]:
from functools import partial
import itertools
import json
import re

import numpy as np
import scipy as sp
import sklearn
from sklearn.utils import check_random_state

from lime import explanation
from lime import lime_base

class OldMyLimeTextExplainer(LimeTextExplainer):
    """Explains text classifiers.
       Currently, we are using an exponential kernel on cosine distance, and
       restricting explanations to words that are present in documents."""

    def __init__(self,
                 kernel_width=25,
                 kernel=None,
                 verbose=False,
                 class_names=None,
                 feature_selection='auto',
                 split_expression=r'\W+',
                 bow=True,
                 mask_string=None,
                 random_state=None,
                 char_level=False,
                 mode='rand'):
        """Init function.
        Args:
            kernel_width: kernel width for the exponential kernel.
            kernel: similarity kernel that takes euclidean distances and kernel
                width as input and outputs weights in (0,1). If None, defaults to
                an exponential kernel.
            verbose: if true, print local prediction values from linear model
            class_names: list of class names, ordered according to whatever the
                classifier is using. If not present, class names will be '0',
                '1', ...
            feature_selection: feature selection method. can be
                'forward_selection', 'lasso_path', 'none' or 'auto'.
                See function 'explain_instance_with_data' in lime_base.py for
                details on what each of the options does.
            split_expression: Regex string or callable. If regex string, will be used with re.split.
                If callable, the function should return a list of tokens.
            bow: if True (bag of words), will perturb input data by removing
                all occurrences of individual words or characters.
                Explanations will be in terms of these words. Otherwise, will
                explain in terms of word-positions, so that a word may be
                important the first time it appears and unimportant the second.
                Only set to false if the classifier uses word order in some way
                (bigrams, etc), or if you set char_level=True.
            mask_string: String used to mask tokens or characters if bow=False
                if None, will be 'UNKWORDZ' if char_level=False, chr(0)
                otherwise.
            random_state: an integer or numpy.RandomState that will be used to
                generate random numbers. If None, the random state will be
                initialized using the internal numpy seed.
            char_level: an boolean identifying that we treat each character
                as an independent occurence in the string
        """

        if kernel is None:
            def kernel(d, kernel_width):
                return np.sqrt(np.exp(-(d ** 2) / kernel_width ** 2))

        kernel_fn = partial(kernel, kernel_width=kernel_width)

        self.random_state = check_random_state(random_state)
        self.base = lime_base.LimeBase(kernel_fn, verbose,
                                       random_state=self.random_state)
        self.class_names = class_names
        self.vocabulary = None
        self.feature_selection = feature_selection
        self.bow = bow
        self.mask_string = mask_string
        self.split_expression = split_expression
        self.char_level = char_level
        self.mode = mode

    def explain_instance(self,
                         text_instance,
                         classifier_fn,
                         labels=(1,),
                         top_labels=None,
                         num_features=10,
                         num_samples=5000,
                         distance_metric='cosine',
                         model_regressor=None):
        """Generates explanations for a prediction.
        First, we generate neighborhood data by randomly hiding features from
        the instance (see __data_labels_distance_mapping). We then learn
        locally weighted linear models on this neighborhood data to explain
        each of the classes in an interpretable way (see lime_base.py).
        Args:
            text_instance: raw text string to be explained.
            classifier_fn: classifier prediction probability function, which
                takes a list of d strings and outputs a (d, k) numpy array with
                prediction probabilities, where k is the number of classes.
                For ScikitClassifiers , this is classifier.predict_proba.
            labels: iterable with labels to be explained.
            top_labels: if not None, ignore labels and produce explanations for
                the K labels with highest prediction probabilities, where K is
                this parameter.
            num_features: maximum number of features present in explanation
            num_samples: size of the neighborhood to learn the linear model
            distance_metric: the distance metric to use for sample weighting,
                defaults to cosine similarity
            model_regressor: sklearn regressor to use in explanation. Defaults
            to Ridge regression in LimeBase. Must have model_regressor.coef_
            and 'sample_weight' as a parameter to model_regressor.fit()
        Returns:
            An Explanation object (see explanation.py) with the corresponding
            explanations.
        """

        indexed_string = (IndexedCharacters(
            text_instance, bow=self.bow, mask_string=self.mask_string)
                          if self.char_level else
                          MyIndexedString(text_instance, bow=self.bow,
                                        split_expression=self.split_expression,
                                        mask_string=self.mask_string))
        domain_mapper = TextDomainMapper(indexed_string)
        data, yss, distances = self.__data_labels_distances(
            indexed_string, classifier_fn, num_samples,
            distance_metric=distance_metric, mode = self.mode)
        if self.class_names is None:
            self.class_names = [str(x) for x in range(yss[0].shape[0])]
        #ret_exp = explanation.Explanation(domain_mapper=domain_mapper,
        ret_exp = MyExplanation(domain_mapper=domain_mapper,
                                          class_names=self.class_names,
                                          random_state=self.random_state)
        ret_exp.predict_proba = yss[0]
        if top_labels:
            labels = np.argsort(yss[0])[-top_labels:]
            ret_exp.top_labels = list(labels)
            ret_exp.top_labels.reverse()
        for label in labels:
            (ret_exp.intercept[label],
             ret_exp.local_exp[label],
             ret_exp.score[label],
             ret_exp.local_pred[label]) = self.base.explain_instance_with_data(
                data, yss, distances, label, num_features,
                model_regressor=model_regressor,
                feature_selection=self.feature_selection)
        return ret_exp

    def __data_labels_distances(self,
                                indexed_string,
                                classifier_fn,
                                num_samples,
                                distance_metric='cosine',
                                mode='rand'):
        """Generates a neighborhood around a prediction.
        Generates neighborhood data by randomly removing words from
        the instance, and predicting with the classifier. Uses cosine distance
        to compute distances between original and perturbed instances.
        Args:
            indexed_string: document (IndexedString) to be explained,
            classifier_fn: classifier prediction probability function, which
                takes a string and outputs prediction probabilities. For
                ScikitClassifier, this is classifier.predict_proba.
            num_samples: size of the neighborhood to learn the linear model
            distance_metric: the distance metric to use for sample weighting,
                defaults to cosine similarity.
        Returns:
            A tuple (data, labels, distances), where:
                data: dense num_samples * K binary matrix, where K is the
                    number of tokens in indexed_string. The first row is the
                    original instance, and thus a row of ones.
                labels: num_samples * L matrix, where L is the number of target
                    labels
                distances: cosine distance between the original instance and
                    each perturbed instance (computed in the binary 'data'
                    matrix), times 100.
        """
        def distance_fn(x):
            return sklearn.metrics.pairwise.pairwise_distances(
                    x, x[0], metric=distance_metric).ravel() * 100

        if not indexed_string.bow: 

            doc_size = indexed_string.num_words()

            sep = indexed_string.return_sep()
            print("sep: ", sep, "docsize: ", doc_size)

            global sample
            sample = self.random_state.randint(1, doc_size + 1, num_samples - 1)

            data = np.ones((num_samples, doc_size))
            data[0] = np.ones(doc_size)
            inverse_data = [indexed_string.raw_string()]
            
            if sep < doc_size:
                print("separation on")

                sample_left = self.random_state.randint(1, sep + 1, num_samples - 1)
                sample_right = self.random_state.randint(1, doc_size - sep, num_samples - 1)

                features_range_left = range(0, sep)
                features_range_right = range(sep, doc_size)

                if mode == 'left':
                    print("left")
                    sample = sample_left
                    features_range = features_range_left

                    #global save_sample
                    #save_sample = sample

                    for i, size in enumerate(sample, start=1):
                        inactive = self.random_state.choice(features_range, size,
                                                        replace=False)
                        data[i, inactive] = 0
                        inverse_data.append(indexed_string.inverse_removing(inactive))

                elif mode == 'right':
                    print("right")
                    sample = sample_right
                    features_range = features_range_right

                    #global save_sample
                    #save_sample = sample

                    for i, size in enumerate(sample, start=1):
                        print(i, len(features_range), size)
                        inactive = self.random_state.choice(features_range, size,
                                                        replace=False)
                        data[i, inactive] = 0
                        inverse_data.append(indexed_string.inverse_removing(inactive))

                else:
                    print('rand')
                    dir = self.random_state.randint(2, size = num_samples - 1)

                    for i, size in enumerate(sample, start=1):
                        print(dir[i-1])
                        if dir[i-1]:
                            inactive = self.random_state.choice(features_range_right, sample_right[i-1],
                                                        replace=False)
                        else:
                            inactive = self.random_state.choice(features_range_left, sample_left[i-1],
                                                        replace=False)
                        data[i, inactive] = 0
                        inverse_data.append(indexed_string.inverse_removing(inactive))

            else:
                features_range = range(doc_size)
                for i, size in enumerate(sample, start=1):
                    inactive = self.random_state.choice(features_range, size,
                                                        replace=False)
                    data[i, inactive] = 0
                    inverse_data.append(indexed_string.inverse_removing(inactive))
        """
        features_range = range(doc_size)
        for i, size in enumerate(sample, start=1):
            inactive = self.random_state.choice(features_range, size,
                                                replace=False)
            data[i, inactive] = 0
            inverse_data.append(indexed_string.inverse_removing(inactive))
        """

        labels = classifier_fn(inverse_data)
        distances = distance_fn(sp.sparse.csr_matrix(data))
        return data, labels, distances

In [ ]:
class MyIndexedString(object):
    """String with various indexes."""

    def __init__(self, raw_string, split_expression=r'\W+', bow=True,
                 mask_string=None):
        """Initializer.
        Args:
            raw_string: string with raw text in it
            split_expression: Regex string or callable. If regex string, will be used with re.split.
                If callable, the function should return a list of tokens.
            bow: if True, a word is the same everywhere in the text - i.e. we
                 will index multiple occurrences of the same word. If False,
                 order matters, so that the same word will have different ids
                 according to position.
            mask_string: If not None, replace words with this if bow=False
                if None, default value is UNKWORDZ
        """
        self.raw = raw_string
        self.mask_string = 'UNKWORDZ' if mask_string is None else mask_string

        if callable(split_expression):
            tokens = split_expression(self.raw)
            self.as_list = self._segment_with_tokens(self.raw, tokens)
            tokens = set(tokens)

            def non_word(string):
                return string not in tokens

        else:
            # with the split_expression as a non-capturing group (?:), we don't need to filter out
            # the separator character from the split results.
            splitter = re.compile(r'(%s)|$' % split_expression)
            self.as_list = [s for s in splitter.split(self.raw) if s]
            non_word = splitter.match

        self.as_np = np.array(self.as_list)
        self.string_start = np.hstack(
            ([0], np.cumsum([len(x) for x in self.as_np[:-1]])))
        vocab = {}
        self.inverse_vocab = []
        self.positions = []
        self.bow = bow

        self.sep = None

        non_vocab = set()
        for i, word in enumerate(self.as_np):
            if "$&*&*&$" in word:
                self.sep = len(self.inverse_vocab)
            if word in non_vocab:
                continue
            if non_word(word):
                non_vocab.add(word)
                continue
            if bow:
                if word not in vocab:
                    vocab[word] = len(vocab)
                    self.inverse_vocab.append(word)
                    self.positions.append([])
                idx_word = vocab[word]
                self.positions[idx_word].append(i)
            else:
                self.inverse_vocab.append(word)
                self.positions.append(i)
        if not bow:
            self.positions = np.array(self.positions)

        if not self.sep:
            self.sep = len(self.inverse_vocab)

    def return_sep(self):
        """Return the index of the separator sequence"""
        return self.sep

    def raw_string(self):
        """Returns the original raw string"""
        return self.raw

    def num_words(self):
        """Returns the number of tokens in the vocabulary for this document."""
        return len(self.inverse_vocab)

    def word(self, id_):
        """Returns the word that corresponds to id_ (int)"""
        return self.inverse_vocab[id_]

    def string_position(self, id_):
        """Returns a np array with indices to id_ (int) occurrences"""
        if self.bow:
            return self.string_start[self.positions[id_]]
        else:
            return self.string_start[[self.positions[id_]]]

    def inverse_removing(self, words_to_remove):
        """Returns a string after removing the appropriate words.
        If self.bow is false, replaces word with UNKWORDZ instead of removing
        it.
        Args:
            words_to_remove: list of ids (ints) to remove
        Returns:
            original raw string with appropriate words removed.
        """
        mask = np.ones(self.as_np.shape[0], dtype='bool')
        mask[self.__get_idxs(words_to_remove)] = False
        if not self.bow:
            return ''.join(
                [self.as_list[i] if mask[i] else self.mask_string
                 for i in range(mask.shape[0])])
        return ''.join([self.as_list[v] for v in mask.nonzero()[0]])

    @staticmethod
    def _segment_with_tokens(text, tokens):
        """Segment a string around the tokens created by a passed-in tokenizer"""
        list_form = []
        text_ptr = 0
        for token in tokens:
            inter_token_string = []
            while not text[text_ptr:].startswith(token):
                inter_token_string.append(text[text_ptr])
                text_ptr += 1
                if text_ptr >= len(text):
                    raise ValueError("Tokenization produced tokens that do not belong in string!")
            text_ptr += len(token)
            if inter_token_string:
                list_form.append(''.join(inter_token_string))
            list_form.append(token)
        if text_ptr < len(text):
            list_form.append(text[text_ptr:])
        return list_form

    def __get_idxs(self, words):
        """Returns indexes to appropriate words."""
        if self.bow:
            return list(itertools.chain.from_iterable(
                [self.positions[z] for z in words]))
        else:
            return self.positions[words]

In [ ]:
from functools import partial
import itertools
import json
import re

import numpy as np
import scipy as sp
import sklearn
from sklearn.utils import check_random_state

from lime import lime_base

from lime.lime_text import TextDomainMapper, LimeTextExplainer

class MyLimeTextExplainer(LimeTextExplainer):
    """Explains text classifiers.
       Currently, we are using an exponential kernel on cosine distance, and
       restricting explanations to words that are present in documents."""

    def __init__(self,
                 kernel_width=25,
                 kernel=None,
                 verbose=False,
                 class_names=None,
                 feature_selection='auto',
                 split_expression=r'\W+',
                 bow=True,
                 mask_string=None,
                 random_state=None,
                 char_level=False,
                 mode='rand'):
        """Init function.
        Args:
            kernel_width: kernel width for the exponential kernel.
            kernel: similarity kernel that takes euclidean distances and kernel
                width as input and outputs weights in (0,1). If None, defaults to
                an exponential kernel.
            verbose: if true, print local prediction values from linear model
            class_names: list of class names, ordered according to whatever the
                classifier is using. If not present, class names will be '0',
                '1', ...
            feature_selection: feature selection method. can be
                'forward_selection', 'lasso_path', 'none' or 'auto'.
                See function 'explain_instance_with_data' in lime_base.py for
                details on what each of the options does.
            split_expression: Regex string or callable. If regex string, will be used with re.split.
                If callable, the function should return a list of tokens.
            bow: if True (bag of words), will perturb input data by removing
                all occurrences of individual words or characters.
                Explanations will be in terms of these words. Otherwise, will
                explain in terms of word-positions, so that a word may be
                important the first time it appears and unimportant the second.
                Only set to false if the classifier uses word order in some way
                (bigrams, etc), or if you set char_level=True.
            mask_string: String used to mask tokens or characters if bow=False
                if None, will be 'UNKWORDZ' if char_level=False, chr(0)
                otherwise.
            random_state: an integer or numpy.RandomState that will be used to
                generate random numbers. If None, the random state will be
                initialized using the internal numpy seed.
            char_level: an boolean identifying that we treat each character
                as an independent occurence in the string
        """

        if kernel is None:
            def kernel(d, kernel_width):
                return np.sqrt(np.exp(-(d ** 2) / kernel_width ** 2))

        kernel_fn = partial(kernel, kernel_width=kernel_width)

        self.random_state = check_random_state(random_state)
        self.base = lime_base.LimeBase(kernel_fn, verbose,
                                       random_state=self.random_state)
        self.class_names = class_names
        self.vocabulary = None
        self.feature_selection = feature_selection
        self.bow = bow
        self.mask_string = mask_string
        self.split_expression = split_expression
        self.char_level = char_level
        self.mode = mode

    def explain_instance(self,
                         text_instance,
                         classifier_fn,
                         labels=(1,),
                         top_labels=None,
                         num_features=10,
                         num_samples=5000,
                         distance_metric='cosine',
                         model_regressor=None):
        """Generates explanations for a prediction.
        First, we generate neighborhood data by randomly hiding features from
        the instance (see __data_labels_distance_mapping). We then learn
        locally weighted linear models on this neighborhood data to explain
        each of the classes in an interpretable way (see lime_base.py).
        Args:
            text_instance: raw text string to be explained.
            classifier_fn: classifier prediction probability function, which
                takes a list of d strings and outputs a (d, k) numpy array with
                prediction probabilities, where k is the number of classes.
                For ScikitClassifiers , this is classifier.predict_proba.
            labels: iterable with labels to be explained.
            top_labels: if not None, ignore labels and produce explanations for
                the K labels with highest prediction probabilities, where K is
                this parameter.
            num_features: maximum number of features present in explanation
            num_samples: size of the neighborhood to learn the linear model
            distance_metric: the distance metric to use for sample weighting,
                defaults to cosine similarity
            model_regressor: sklearn regressor to use in explanation. Defaults
            to Ridge regression in LimeBase. Must have model_regressor.coef_
            and 'sample_weight' as a parameter to model_regressor.fit()
        Returns:
            An Explanation object (see explanation.py) with the corresponding
            explanations.
        """

        indexed_string = (IndexedCharacters(
            text_instance, bow=self.bow, mask_string=self.mask_string)
                          if self.char_level else
                          MyIndexedString(text_instance, bow=self.bow,
                                        split_expression=self.split_expression,
                                        mask_string=self.mask_string))
        domain_mapper = TextDomainMapper(indexed_string)
        data, yss, distances = self.__data_labels_distances(
            indexed_string, classifier_fn, num_samples,
            distance_metric=distance_metric, mode = self.mode)
        if self.class_names is None:
            self.class_names = [str(x) for x in range(yss[0].shape[0])]
        #ret_exp = explanation.Explanation(domain_mapper=domain_mapper,
        ret_exp = MyExplanation(domain_mapper=domain_mapper,
                                          class_names=self.class_names,
                                          random_state=self.random_state)
        ret_exp.predict_proba = yss[0]
        if top_labels:
            labels = np.argsort(yss[0])[-top_labels:]
            ret_exp.top_labels = list(labels)
            ret_exp.top_labels.reverse()
        for label in labels:
            (ret_exp.intercept[label],
             ret_exp.local_exp[label],
             ret_exp.score[label],
             ret_exp.local_pred[label]) = self.base.explain_instance_with_data(
                data, yss, distances, label, num_features,
                model_regressor=model_regressor,
                feature_selection=self.feature_selection)
        return ret_exp

    def __data_labels_distances(self,
                                indexed_string,
                                classifier_fn,
                                num_samples,
                                distance_metric='cosine',
                                mode='rand'):
        """Generates a neighborhood around a prediction.
        Generates neighborhood data by randomly removing words from
        the instance, and predicting with the classifier. Uses cosine distance
        to compute distances between original and perturbed instances.
        Args:
            indexed_string: document (IndexedString) to be explained,
            classifier_fn: classifier prediction probability function, which
                takes a string and outputs prediction probabilities. For
                ScikitClassifier, this is classifier.predict_proba.
            num_samples: size of the neighborhood to learn the linear model
            distance_metric: the distance metric to use for sample weighting,
                defaults to cosine similarity.
        Returns:
            A tuple (data, labels, distances), where:
                data: dense num_samples * K binary matrix, where K is the
                    number of tokens in indexed_string. The first row is the
                    original instance, and thus a row of ones.
                labels: num_samples * L matrix, where L is the number of target
                    labels
                distances: cosine distance between the original instance and
                    each perturbed instance (computed in the binary 'data'
                    matrix), times 100.
        """
        def distance_fn(x):
            return sklearn.metrics.pairwise.pairwise_distances(
                    x, x[0], metric=distance_metric).ravel() * 100

        if not indexed_string.bow: 

            doc_size = indexed_string.num_words()

            sep = indexed_string.return_sep()
            print("sep: ", sep, "docsize: ", doc_size)

            global sample
            sample = self.random_state.randint(1, doc_size + 1, num_samples - 1)

            data = np.ones((num_samples, doc_size))
            data[0] = np.ones(doc_size)
            inverse_data = [indexed_string.raw_string()]
            
            if sep < doc_size:
                print("separation on")

                sample_left = self.random_state.randint(1, sep + 1, num_samples - 1)
                sample_right = self.random_state.randint(1, doc_size - sep, num_samples - 1)

                features_range_left = range(0, sep)
                features_range_right = range(sep, doc_size)

                if mode == 'left':
                    print("left")
                    sample = sample_left
                    features_range = features_range_left

                    #global save_sample
                    #save_sample = sample

                    for i, size in enumerate(sample, start=1):
                        inactive = self.random_state.choice(features_range, size,
                                                        replace=False)
                        data[i, inactive] = 0
                        inverse_data.append(indexed_string.inverse_removing(inactive))

                elif mode == 'right':
                    print("right")
                    sample = sample_right
                    features_range = features_range_right

                    #global save_sample
                    #save_sample = sample

                    for i, size in enumerate(sample, start=1):
                        print(i, len(features_range), size)
                        inactive = self.random_state.choice(features_range, size,
                                                        replace=False)
                        data[i, inactive] = 0
                        inverse_data.append(indexed_string.inverse_removing(inactive))

                else:
                    print('rand')
                    dir = self.random_state.randint(2, size = num_samples - 1)

                    for i, size in enumerate(sample, start=1):
                        #print(dir[i-1])
                        if dir[i-1]:
                            inactive = self.random_state.choice(features_range_right, sample_right[i-1],
                                                        replace=False)
                        else:
                            inactive = self.random_state.choice(features_range_left, sample_left[i-1],
                                                        replace=False)
                        data[i, inactive] = 0
                        inverse_data.append(indexed_string.inverse_removing(inactive))

            else:
                features_range = range(doc_size)
                for i, size in enumerate(sample, start=1):
                    inactive = self.random_state.choice(features_range, size,
                                                        replace=False)
                    data[i, inactive] = 0
                    inverse_data.append(indexed_string.inverse_removing(inactive))
        """
        features_range = range(doc_size)
        for i, size in enumerate(sample, start=1):
            inactive = self.random_state.choice(features_range, size,
                                                replace=False)
            data[i, inactive] = 0
            inverse_data.append(indexed_string.inverse_removing(inactive))
        """

        labels = classifier_fn(inverse_data)
        distances = distance_fn(sp.sparse.csr_matrix(data))
        return data, labels, distances